# Quickstart for Customer Churn Model Documentation — Full Suite

This interactive notebook guides model developers through the process of documenting a model with the ValidMind Developer Framework. It uses the [Bank Customer Churn Prediction](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from Kaggle to train a simple classification model.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train a simple classification model
- Running a ValidMind test suite to quickly generate documention about the data and model


## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).


## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [2]:
## Replace this placeholder with the code snippet from your own project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

2024-03-08 17:18:42,915 - INFO(validmind.api_client): Connected to ValidMind. Project: [Test] Customer Churn - Initial Validation (cltdu833a00058e8hwiums2mj)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:


In [3]:
import xgboost as xgb

%matplotlib inline

### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:


In [4]:
vm.preview_template()

2024-03-08 17:18:44,382 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierInSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierInSamplePerformance'
2024-03-08 17:18:44,385 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierOutOfSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierOutOfSamplePerformance'


Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


In [5]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()
raw_df.head()

Loaded demo dataset with: 

	• Target column: 'Exited' 
	• Class labels: {'0': 'Did not exit', '1': 'Exited'}


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.


### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize XGBoost classifier: Creates an `XGBClassifier` object with early stopping rounds set to 10.
- Set evaluation metrics: Specifies metrics for model evaluation as "error," "logloss," and "auc."
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.


In [6]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset
- `class_labels` — an optional value to map predicted classes to class labels

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [7]:
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df, input_id="train_dataset", target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, input_id="test_dataset", target_column=demo_dataset.target_column
)

2024-03-08 17:18:47,113 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-03-08 17:18:47,249 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-03-08 17:18:47,347 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Initialize a model object

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):


In [8]:
vm_model = vm.init_model(
    model,
    input_id="model",
)

### Assign predictions to the datasets

We can now use the assign_predictions() method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [9]:
vm_train_ds.assign_predictions(
    model=vm_model,
)

vm_test_ds.assign_predictions(
    model=vm_model,
)

2024-03-08 17:18:47,469 - INFO(validmind.vm_models.dataset): Running predict()... This may take a while
2024-03-08 17:18:47,474 - INFO(validmind.vm_models.dataset): Running predict()... This may take a while


### Run the full suite of tests

This is where it all comes together: you are now ready to run the documentation tests for the model as defined by the documentation template you looked at earlier.

The [`vm.run_documentation_tests`](https://docs.validmind.ai/validmind/validmind.html#run_documentation_tests) function finds and runs every test specified in the template and then uploads all the documentation and test artifacts that get generated to the ValidMind AI Risk Platform.

The function takes a `config` argument that has information about the `params` and `inputs` that each test requires. The method `get_input_config()` below returns the default input configuration for our demo customer churn model.

The variable `full_suite` then holds the result of these tests.


In [10]:
test_config = demo_dataset.get_test_config(
    vm,
    {
        "raw_dataset": vm_raw_dataset,
        "train_dataset": vm_train_ds,
        "test_dataset": vm_test_ds,
        "model": vm_model,
    },
)

full_suite = vm.run_documentation_tests(config=test_config)

2024-03-08 17:18:47,538 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierInSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierInSamplePerformance'
2024-03-08 17:18:47,540 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierOutOfSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierOutOfSamplePerformance'
2024-03-08 17:18:47,542 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierInSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierInSamplePerformance'
2024-03-08 17:18:47,543 - ERROR(validmind.tests): Unable to load test validmind.model_validation.sklearn.ClassifierOutOfSamplePerformance. No module named 'validmind.tests.model_validation.sklearn.ClassifierOutOfSamplePerformance'


2024-03-08 17:18:48,060 - ERROR(validmind.vm_models.test_suite.test): Failed to run test 'iqr_outliers_bar_plot': (AttributeError) 'TestInput' object has no attribute 'dataset'


## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI:

1. Log back into the [Platform UI](https://app.prod.validmind.ai)

2. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Documentation**.

3. Expand the following sections and take a look around:

   - **2. Data Preparation**
   - **3. Model Development**

What you can see now is a much more easily consumable version of the documentation, including the results of the tests you just performed, along with other parts of your documentation project that still need to be completed. There is a wealth of information that gets uploaded when you run the full test suite, so take a closer look around, especially at test results that might need attention (hint: some of the tests in 2.1 Data description look like they need some attention).

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).
